# Analyzing CEX Price Impact using CCXT

Here we extract CCXT `trade` and `ohlcv` data to approximate the price impact of trades against centralized exchanges.

In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
import ccxt as ccxt
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from src.utils.ccxtdatafetcher import CCXTDataFetcher
import asyncio

load_dotenv()
coinbase_pro_api_key = os.getenv('COINBASE_PRO_API_KEY')
coinbase_pro_api_secret = os.getenv('COINBASE_PRO_API_SECRET')
coinbase_pro_api_password = os.getenv('COINBASE_PRO_API_PWD')

In [7]:
datafetcher = CCXTDataFetcher(coinbase_pro_api_key, coinbase_pro_api_secret, coinbase_pro_api_password)

In [13]:
df = pd.read_parquet('../data/trades/binance_ETH_USDC_1699592400.gzip')
df.head()

,info,timestamp,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,fees
datetime,,,,,,,,,,,,,
2023-11-10T00:00:05.141Z,"{'M': True, 'T': '1699574405141', 'a': '307378...",1699574405141,ETH/USDC,30737891,None,None,buy,None,2121.37,0.3485,739.297445,None,[]
2023-11-10T00:00:05.142Z,"{'M': True, 'T': '1699574405142', 'a': '307378...",1699574405142,ETH/USDC,30737892,None,None,buy,None,2121.37,2.0084,4260.559508,None,[]
2023-11-10T00:00:06.595Z,"{'M': True, 'T': '1699574406595', 'a': '307378...",1699574406595,ETH/USDC,30737893,None,None,buy,None,2121.65,1.2453,2642.090745,None,[]
2023-11-10T00:00:06.595Z,"{'M': True, 'T': '1699574406595', 'a': '307378...",1699574406595,ETH/USDC,30737894,None,None,buy,None,2121.65,1.1113,2357.789645,None,[]
2023-11-10T00:00:09.442Z,"{'M': True, 'T': '1699574409442', 'a': '307378...",1699574409442,ETH/USDC,30737895,None,None,buy,None,2121.66,0.9629,2042.946414,None,[]


In [14]:
df.shape

(24834, 13)

In [ ]:
symbol = "ETH/USDC"
exchange = datafetcher.coinbasepro

ohlcv = exchange.fetch_ohlcv(symbol, timeframe="1h")
ohlcv

# Archived code for using async ccxt

In [ ]:
# This is failing for some reason

exchange = ccxt.coinbasepro({
    "apiKey": coinbase_pro_api_key,
    "secret": coinbase_pro_api_secret,
    "password": coinbase_pro_api_password,
    "enableRateLimit": True,
})

symbol = "ETH/USD"

async def fetch(exchange, page):
    try:
        trades = await exchange.fetch_trades(
            symbol, params={'after': page}
        )
    except ccxt.RateLimitExceeded as e:
        print('Sleeping')
        await exchange.sleep(10000)
        return await fetch(exchange, page)
    except Exception as e:
        raise e

async def fetch_all(exchange, symbol):
    # Determine how to batch async requests
    trades = await exchange.fetch_trades(symbol)

    # Let's try to do like 10% of the expected pages at a time?
    since = exchange.parse8601('2023-11-09T00:00:00Z')
    expected_range = trades[-1]['timestamp'] - trades[0]['timestamp']
    requested_range = trades[-1]['timestamp'] - since
    expected_pages = int(requested_range / expected_range)

    batch_size = min(2000, expected_pages)

    first_page = int(exchange.last_response_headers.get('Cb-After'))
    pages = [first_page - 1000 * step for step in range(batch_size)]

    tasks = (asyncio.create_task(fetch(exchange, page)) for page in pages)
    results = await asyncio.gather(*tasks)
    return results

async def main():
    results = await fetch_all(exchange, symbol)
    print(results)
    return results

results = await main()
await exchange.close()